<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Preparation_des_donnees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création de données d'entrée X avec l'utilisation d'une fenêtre glissante

**1. Création d'une liste de données**

Commençons par créer un ensemble de 10 valeurs avec l'outils `Dataset` de tensorflow :

In [12]:
import tensorflow as tf
import numpy as np

# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

for valeur in dataset:
  print("%02d" %valeur.numpy())

00
01
02
03
04
05
06
07
08
09
10
11
12
13
14


**2. Extraction des sous-ensembles à l'aide d'une fenêtre glissante**

Utilisons maintenant l'outils `window` de la classe `Dataset` afin d'obtenir plusieurs sous-ensembles. Ces sous-ensembles sont créés à l'aide d'une fenêtre glissante appliquée sur les valeurs précédentes. Les paramètres de la fenêtre glissante sont :
 - Taille de la fenêtre : 5
 - Décalage : 1


In [13]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
dataset = dataset.window(5,shift=1)

i=0
for fenetre in dataset:
  print ("Fenêtre #%02d :" %i, end=" ")
  for valeur in fenetre:
    print ("%02d" %valeur.numpy(), end=" ")
  print()
  i = i+1

Fenêtre #00 : 00 01 02 03 04 
Fenêtre #01 : 01 02 03 04 05 
Fenêtre #02 : 02 03 04 05 06 
Fenêtre #03 : 03 04 05 06 07 
Fenêtre #04 : 04 05 06 07 08 
Fenêtre #05 : 05 06 07 08 09 
Fenêtre #06 : 06 07 08 09 10 
Fenêtre #07 : 07 08 09 10 11 
Fenêtre #08 : 08 09 10 11 12 
Fenêtre #09 : 09 10 11 12 13 
Fenêtre #10 : 10 11 12 13 14 
Fenêtre #11 : 11 12 13 14 
Fenêtre #12 : 12 13 14 
Fenêtre #13 : 13 14 
Fenêtre #14 : 14 


Faisons maintenant en sorte que chaque fenêtre possède le même nombre de données (5 données), en éliminant les fenêtres trop petites :

In [14]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

i=0
for fenetre in dataset:
  print ("Fenêtre #%02d :" %i, end=" ")
  for valeur in fenetre:
    print ("%02d" %valeur.numpy(),end=" ")
  print()
  i=i+1

Fenêtre #00 : 00 01 02 03 04 
Fenêtre #01 : 01 02 03 04 05 
Fenêtre #02 : 02 03 04 05 06 
Fenêtre #03 : 03 04 05 06 07 
Fenêtre #04 : 04 05 06 07 08 
Fenêtre #05 : 05 06 07 08 09 
Fenêtre #06 : 06 07 08 09 10 
Fenêtre #07 : 07 08 09 10 11 
Fenêtre #08 : 08 09 10 11 12 
Fenêtre #09 : 09 10 11 12 13 
Fenêtre #10 : 10 11 12 13 14 


**3. Conversion des sous-ensembles en vecteurs numpy**

Commençons par regarder comment obtenir les valeurs contenues dans chaque fenêtre sous forme de vecteur à l'aide de la fonction `batch()` :

In [15]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    print(list(vecteurs.as_numpy_iterator()))


[array([0, 1, 2, 3, 4])]
[array([1, 2, 3, 4, 5])]
[array([2, 3, 4, 5, 6])]
[array([3, 4, 5, 6, 7])]
[array([4, 5, 6, 7, 8])]
[array([5, 6, 7, 8, 9])]
[array([ 6,  7,  8,  9, 10])]
[array([ 7,  8,  9, 10, 11])]
[array([ 8,  9, 10, 11, 12])]
[array([ 9, 10, 11, 12, 13])]
[array([10, 11, 12, 13, 14])]


Convertissons maintenant chaque vecteur au format numpy avec la fonction `.numpy()` ou bien en utilisant la fonction `asarray()` de numpy

In [16]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    for vecteur in vecteurs:
      print(np.asarray(vecteur))
#      print(vecteur.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]
[ 6  7  8  9 10]
[ 7  8  9 10 11]
[ 8  9 10 11 12]
[ 9 10 11 12 13]
[10 11 12 13 14]


Un manière plus élégante est d'utiliser les fonctions lambda avec la fonction d'applatissement `flat_map` de tensorflow :

In [17]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Applique la fonction lambda sur le dataset et applatit le résultat
vecteurs = dataset.flat_map(lambda x: x.batch(5))

for vecteur in vecteurs:
  print(vecteur.numpy())


[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]
[ 6  7  8  9 10]
[ 7  8  9 10 11]
[ 8  9 10 11 12]
[ 9 10 11 12 13]
[10 11 12 13 14]


Nous avons maintenant nos données d'entrées X au format Numpy !

# Création des labels Y à partir des données d'entrées X

Il faut maintenant construire les labels Y à partir des données d'entrées X. Choisissons par exemple que le label associé à chaque ensemble d'entrée soit la prochaine valeur à venir de  cet ensemble :

<img src="https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/images/X_Y.png?raw=true" width="300">